In [1]:
from HUB import HUB_DAMAC

In [2]:
laboral = HUB_DAMAC.mercado_laboral()

In [3]:
%config Completer.use_jedi = False

In [4]:
laboral.indicadores_mercado_laboral

,Indicador,Frecuencia,Fuente
0,Tasa de desempleo,Mensual,https://totoro.banrep.gov.co/estadisticas-econ...
1,Tasa de ocupación,Mensual,https://totoro.banrep.gov.co/estadisticas-econ...
2,Tasa global de participación,Mensual,https://totoro.banrep.gov.co/estadisticas-econ...
3,Informalidad,Trimestral,https://www.dane.gov.co/index.php/estadisticas...
4,Desempleo_desestacionalizado,Mensual,https://www.dane.gov.co/index.php/estadisticas...
5,Desempleo_por_sexo,Trimestre móvil,https://www.dane.gov.co/index.php/estadisticas...
6,Desempleo_por_region,Semestral,https://www.dane.gov.co/index.php/estadisticas...


In [5]:
laboral.actualizar(carpeta=r"D:\Desktop\Mercado Laboral",actualizar_todo=True,
                   excel = True,hipervinculos=True,t=15)

C:\Programas\Miniconda\envs\hubdamac\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [1]:
import pandas as pd
import numpy as np

from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook

import os
import xlrd
import shutil


path = r"D:\Desktop\prueba"


            

        try:
            archivos =  pd.Series(os.listdir(path))
            dane_informalidad_nombre = archivos[archivos.str.contains('informalidad')].values[0]
            for i in archivos:
                if i ==  dane_informalidad_nombre:
                    try:
                        data = load_workbook(path+"\{}".format(i))
                        sheets = data.sheetnames
                    except:
                        data = xlrd.open_workbook_xls(path+"\{}".format(i))
                        sheets = sheets = data.sheet_names()


                    #Prop informalidad Total Nacional
                    try:
                        sheets = pd.Series(sheets).str.lower().str.replace(' ','')
                        tnal_nacional_informalidad_index = sheets[sheets.str.contains('informalidad')].index[0]

                        df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_informalidad_index)

                        index = df[df.iloc[:,0].str.contains('23 Ciudades').fillna(False)].index[0]

                        data_23_ciudades = df.iloc[index:,:]

                        periodo = data_23_ciudades[data_23_ciudades.iloc[:,1].str.contains('Ene-').fillna(False)].dropna(axis=1).T
                        periodo.columns = ['Periodo']
                        periodo.reset_index(level=0,drop=True,inplace = True)

                        fecha = pd.date_range(start='2007',freq='M',periods=len(periodo),name = 'Fecha')
                        periodo['Fecha'] = fecha

                        tasa = data_23_ciudades[data_23_ciudades.iloc[:,0].str.contains('Ocupados').fillna(False)].dropna(axis=1).T.iloc[1:,:]

                        tasa.columns = ['Ocupados Informales']
                        tasa['Ocupados Informales'] = tasa['Ocupados Informales'].astype('float')
                        tasa.reset_index(level=0,drop=True,inplace=True)

                        tasa_informalidad = pd.concat([periodo,tasa],axis=1)
                        tasa_informalidad.to_csv(path+'\informalidad_total_Nacional.csv',sep=';',decimal=',',index=False)
                    except:
                        print('La propoción de informalidad total nacional no se pudo limpiar correctamente')
                        pass


                    #Informalidad Ciudad
                    try:
                        sheets = pd.Series(sheets).str.lower().str.replace(' ','')
                        tnal_nacional_ciudad_index = sheets[sheets.str.contains('ciudades')].index[0]

                        df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_ciudad_index)

                        l = df.iloc[:,0]
                        sup = l[l.str.contains('Total 13 ciudades y AM').fillna(False)].index[-1]
                        inf = l[l.str.contains('23 ciudades y áreas').fillna(False)].index[-1]

                        df = df.iloc[sup:inf+5,:]

                        df.reset_index(level=0,drop=True,inplace=True)

                        ind = [i.lower().replace(' ','_') for i in pd.Series("""Ocupados
                        Formales
                        Informales""").str.split('\n')[0]]

                        dic = pd.DataFrame({})
                        for j in ind:
                            # Total Nacional
                            ser_index_nac = df[df.applymap(lambda x: str(x).lower().replace(' ','_') == 'informales')].dropna(how='all',axis=0).index
                        for ix in ser_index_nac:
                            ser_ = df.iloc[ix-4:ix+1,0:]
                            ser_ = ser_.T
                            ser_.columns = ser_.iloc[0,:]
                            ser_ = ser_.reset_index(level=0,drop=True).drop([0],axis=0)
                            fecha = pd.date_range(start='2007-01-01',periods=len(ser_),freq='M',name='Fecha')
                            ser_ = ser_.set_index(fecha,drop=True)
                            ser_['Ocupados'] = ser_['Ocupados'].astype('float')*1000
                            ser_['Formales'] = ser_['Formales'].astype('float')*1000
                            ser_['Informales'] = ser_['Informales'].astype('float')*1000

                            ser_ = ser_.rename(columns={'Ocupados':'Ocupados_{}'.format(ser_.columns[0][:3]),
                                                        'Formales':'Formales_{}'.format(ser_.columns[0][:3]),
                                                        'Informales':'Informales_{}'.format(ser_.columns[0][:3])
                                                        })

                            ser_ = ser_.fillna(method='ffill')

                            dic['Trimestre Móvil'] = ser_.iloc[:,1]

                            dic[ser_.columns[0]] = ser_.iloc[:,0].apply(lambda x: str(x).replace(str(x),ser_.columns[0]))
                            dic[ser_.columns[2]] = ser_.iloc[:,2].apply(lambda x: float(x))
                            dic[ser_.columns[3]] = ser_.iloc[:,3].apply(lambda x: float(x))
                            dic[ser_.columns[4]] = ser_.iloc[:,4].apply(lambda x: float(x))

                        dic.to_csv(path+r"\informalidad_ciudades.csv",sep=';',decimal=',',encoding='utf-8')
                    except:
                        print("La informalidad por Ciudades no se pudo limpiar correctamente")
                        pass

                    #Informalidad por sexo
                    try:
                        sheets = pd.Series(sheets).str.lower().str.replace(' ','')
                        tnal_nacional_sexo_index = sheets[sheets.str.contains('sexo')].index[0]
                        df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_sexo_index)

                        ciudades = ["Ocupados 13 ciudades y áreas metropolitanas","Ocupados 23 ciudades y áreas metropolitanas"]

                        for j in ciudades:
                            l = df.iloc[:,0]
                            sup = l[l.str.contains(j).fillna(False)].index[0]

                            df_temp = df.iloc[sup:sup+9,:]
                            df_temp = df_temp.T.dropna(how='all',axis=0)
                            df_temp.columns = df_temp.iloc[0,:]

                            df_temp.reset_index(level=0,drop=True,inplace=True)
                            df_temp = df_temp.drop([0],axis=0)

                            fecha = pd.date_range(start='2007-01-01',periods=len(df_temp),freq='M',name='Fecha')

                            df_temp.set_index(fecha,drop=True,inplace=True)
                            df_temp = df_temp.applymap(lambda x: float(x)*1000)

                            df_temp.to_csv(path+r"\informalidad_sexo_{}.csv".format(j),sep=';',decimal=',',encoding='utf-8')
                    except:
                        print("La informalidad por sexo no se pudo limpiar correctamente")
                        pass
                    ## Imformalidad por educacion

                    try:
                        sheets = pd.Series(sheets).str.lower().str.replace(' ','')
                        tnal_nacional_educacion_index = sheets[sheets.str.contains('educación')].index[0]
                        df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_educacion_index)

                        ciudades = ["Ocupados 13 ciudades y áreas metropolitanas","Ocupados 23 ciudades y áreas metropolitanas"]

                        for j in ciudades:
                            l = df.iloc[:,0]
                            sup = l[l.str.contains(j).fillna(False)].index[0] 
                            df_temp = df.iloc[sup:sup+18,:]
                            df_temp = df_temp.T.dropna(how='all',axis=0)
                            df_temp.columns = df_temp.iloc[0,:]

                            df_temp.reset_index(level=0,drop=True,inplace=True)
                            df_temp = df_temp.drop([0],axis=0)

                            fecha = pd.date_range(start='2007-01-01',periods=len(df_temp),freq='M',name='Fecha')

                            df_temp.set_index(fecha,drop=True,inplace=True)
                            df_temp = df_temp.applymap(lambda x: float(x)*1000)

                            df_temp.to_csv(path+r"\informalidad_educacion_{}.csv".format(j),sep=';',decimal=',',encoding='utf-8')

                    except:
                        print("La informalidad por educacion no se pudo limpiar correctamente")
                        pass


                    ## Informalidad por ramas ciiu 4a
                    try:
                        sheets = pd.Series(sheets).str.lower().str.replace(' ','')
                        tnal_nacional_educacion_index = sheets[sheets.str.contains('ciiu4')].index[0]
                        df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_educacion_index)

                        ciudades = ["Total 13 áreas","Ocupados 23 ciudades y áreas metropolitanas"]

                        for j in ciudades:
                            l = df.iloc[:,0]
                            sup = l[l.str.contains(j).fillna(False)].index[0] 
                            df_temp = df.iloc[sup:sup+48,:]
                            df_temp = df_temp.T.dropna(how='all',axis=0)
                            df_temp.columns = df_temp.iloc[0,:]

                            df_temp.reset_index(level=0,drop=True,inplace=True)
                            df_temp = df_temp.drop([0],axis=0)

                            fecha = pd.date_range(start='2007-01-01',periods=len(df_temp),freq='M',name='Fecha')

                            df_temp.set_index(fecha,drop=True,inplace=True)
                            df_temp = df_temp.applymap(lambda x: float(x)*1000)

                            df_temp.to_csv(path+r"\informalidad_ramasciiu4a_{}.csv".format(j),sep=';',decimal=',',encoding='utf-8')
                    except:
                        print("La informalidad por ramas CIIU4a no se pudo limpiar corectamente")
                        pass

                    #Informalidad por Seguridad Social
                    try:
                        sheets = pd.Series(sheets).str.lower().str.replace(' ','')
                        tnal_nacional_educacion_index = sheets[sheets.str.contains('seguridadsocial13')].index[0]
                        df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_educacion_index)

                        ciudades = ["Ocupados 13 ciudades y áreas metropolitanas","Ocupados 23 ciudades y áreas metropolitanas"]


                        #Cantidad de personas
                        for j in ciudades:
                            l = df.iloc[:,0]
                            sup = l[l.str.contains(j).fillna(False)].index[0] 
                            df_temp = df.iloc[sup:sup+10,:]
                            df_temp = df_temp.T.dropna(how='all',axis=0)
                            df_temp.columns = df_temp.iloc[0,:]

                            df_temp.reset_index(level=0,drop=True,inplace=True)
                            df_temp = df_temp.drop([0],axis=0)

                            fecha = pd.date_range(start='2007-01-01',periods=len(df_temp),freq='M',name='Fecha')

                            df_temp.set_index(fecha,drop=True,inplace=True)
                            df_temp = df_temp.applymap(lambda x: float(x)*1000)

                            df_temp.to_csv(path+r"\informalidad_segsocial_cantidad_{}.csv".format(j),sep=';',decimal=',',encoding='utf-8')

                        #Porcentaje de personas
                        for j in ciudades:
                            l = df.iloc[:,0]
                            sup = l[l.str.contains(j).fillna(False)].index[1] 
                            df_temp = df.iloc[sup:sup+10,:]
                            df_temp = df_temp.T.dropna(how='all',axis=0)
                            df_temp.columns = df_temp.iloc[0,:]

                            df_temp.reset_index(level=0,drop=True,inplace=True)
                            df_temp = df_temp.drop([0],axis=0)

                            fecha = pd.date_range(start='2007-01-01',periods=len(df_temp),freq='M',name='Fecha')

                            df_temp.set_index(fecha,drop=True,inplace=True)
                            df_temp = df_temp.applymap(lambda x: float(x))

                            df_temp.to_csv(path+r"\informalidad_segsocial_porcentaje_{}.csv".format(j),sep=';',decimal=',',encoding='utf-8')
                    except:
                        print("La informalidad por seguridad social no se pudo limpiar correctamente")
                        pass

            shutil.move(path+r'\{}'.format(dane_informalidad_nombre),path+r'\archivos_fuente\{}'.format(dane_informalidad_nombre))
        except:
            pass